In [1]:
import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

In [3]:
train.head()

,Won_Championship,Previous_SB_Wins,Number_Of_Wins_This_Season,Number_Of_First_Round_Draft_Picks,Team_Value,Playing_Style,Average_Player_Age,Number_Of_Injured_Players,Coach_Experience_Level,ID
0,0,3,13,2,Less_Than_Four_Billion,Balanced,27,five,Intermediate,6056
1,0,2,14,2,Less_Than_Four_Billion,Aggressive_Offense,26,five,Intermediate,9702
2,1,2,13,1,Less_Than_Four_Billion,Aggressive_Defense,27,four,Intermediate,1745
3,0,2,12,2,Above_Four_Billion,Balanced,27,six,Intermediate,4757
4,0,1,15,2,Less_Than_Four_Billion,Balanced,26,five,Intermediate,7242


In [4]:
train['Won_Championship'].value_counts()

1    3255
0    3245
Name: Won_Championship, dtype: int64

In [5]:
test_id = test['ID']

In [6]:
test = test.drop(['ID', 'Playing_Style'], axis=1)

In [7]:
train = train.drop(['ID', 'Playing_Style'], axis=1)

In [8]:
train.head()

,Won_Championship,Previous_SB_Wins,Number_Of_Wins_This_Season,Number_Of_First_Round_Draft_Picks,Team_Value,Average_Player_Age,Number_Of_Injured_Players,Coach_Experience_Level
0,0,3,13,2,Less_Than_Four_Billion,27,five,Intermediate
1,0,2,14,2,Less_Than_Four_Billion,26,five,Intermediate
2,1,2,13,1,Less_Than_Four_Billion,27,four,Intermediate
3,0,2,12,2,Above_Four_Billion,27,six,Intermediate
4,0,1,15,2,Less_Than_Four_Billion,26,five,Intermediate


In [9]:
train['level+Value'] = train['Coach_Experience_Level'] + '_' + train['Team_Value']
test['level+Value'] = test['Coach_Experience_Level'] + '_' + test['Team_Value']

In [10]:
train = train.drop('Team_Value', axis=1)
test = test.drop('Team_Value', axis=1)

In [11]:
X = train.drop('Won_Championship', axis=1)
y = train['Won_Championship']

In [12]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [13]:

from sklearn.model_selection import train_test_split, StratifiedKFold

In [14]:
# Split the data into 30% validation and 70% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1234)

In [15]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [16]:
model = CatBoostClassifier(#reg_lambda=5,#learning_rate=0.1,#200 best so far
                          loss_function='Logloss', eval_metric='F1',
                           #learning_rate=0.03,
                       metric_period=20)#5

In [17]:
#model.fit(X_train, y_train, cat_features=categorical_features_indices)

In [18]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score, mean_absolute_error, mean_squared_error
errcb=[]
y_pred_totcb=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit, GroupKFold
fold=StratifiedKFold(n_splits=40,shuffle=True, random_state=101)#101
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index],
    y_train, y_test = y[train_index], y[test_index]
    m=model#LGBMRegressor(**params)
    #LGBMRegressor(**params)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)],cat_features=categorical_features_indices, early_stopping_rounds=100)
    preds=m.predict(X_test)
    print("err: ",accuracy_score(y_test,preds))
    errcb.append(accuracy_score(y_test,preds))
    p = m.predict(test)
    y_pred_totcb.append(p)

Learning rate set to 0.09472
0:	learn: 0.5425085	test: 0.5438139	test1: 0.5245902	best: 0.5245902 (0)	total: 75.8ms	remaining: 1m 15s


20:	learn: 0.7558656	test: 0.7647990	test1: 0.7654321	best: 0.7672956 (12)	total: 304ms	remaining: 14.2s
40:	learn: 0.7602882	test: 0.7688903	test1: 0.7804878	best: 0.7804878 (38)	total: 556ms	remaining: 13s
60:	learn: 0.7621795	test: 0.7713386	test1: 0.7804878	best: 0.7804878 (38)	total: 800ms	remaining: 12.3s
80:	learn: 0.7676022	test: 0.7738152	test1: 0.7878788	best: 0.7878788 (80)	total: 1.07s	remaining: 12.2s
100:	learn: 0.7709569	test: 0.7730959	test1: 0.7804878	best: 0.7878788 (80)	total: 1.38s	remaining: 12.3s
120:	learn: 0.7731200	test: 0.7754002	test1: 0.7730061	best: 0.7878788 (80)	total: 1.74s	remaining: 12.6s
140:	learn: 0.7757382	test: 0.7752773	test1: 0.7730061	best: 0.7878788 (80)	total: 2.12s	remaining: 12.9s
160:	learn: 0.7784507	test: 0.7757883	test1: 0.7730061	best: 0.7878788 (80)	total: 2.42s	remaining: 12.6s
180:	learn: 0.7806041	test: 0.7755619	test1: 0.7804878	best: 0.7878788 (80)	total: 2.77s	remaining: 12.6s
Stopped by overfitting detector  (100 iterations wai

20:	learn: 0.7516996	test: 0.7584088	test1: 0.7019868	best: 0.7225806 (18)	total: 282ms	remaining: 13.2s
40:	learn: 0.7619048	test: 0.7698222	test1: 0.7284768	best: 0.7320261 (27)	total: 505ms	remaining: 11.8s
60:	learn: 0.7668377	test: 0.7734139	test1: 0.7354839	best: 0.7354839 (45)	total: 729ms	remaining: 11.2s
80:	learn: 0.7701903	test: 0.7751816	test1: 0.7354839	best: 0.7354839 (45)	total: 995ms	remaining: 11.3s
100:	learn: 0.7725747	test: 0.7751226	test1: 0.7272727	best: 0.7354839 (45)	total: 1.32s	remaining: 11.8s
120:	learn: 0.7768727	test: 0.7703442	test1: 0.7152318	best: 0.7354839 (45)	total: 1.66s	remaining: 12s
140:	learn: 0.7793651	test: 0.7699984	test1: 0.7152318	best: 0.7354839 (45)	total: 2.04s	remaining: 12.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.735483871
bestIteration = 45

Shrink model to first 46 iterations.
err:  0.75
Learning rate set to 0.09472
0:	learn: 0.5424614	test: 0.5430809	test1: 0.5538462	best: 0.5538462 (0)	total: 21ms	rem

20:	learn: 0.7609833	test: 0.7677844	test1: 0.7836257	best: 0.7836257 (16)	total: 283ms	remaining: 13.2s
40:	learn: 0.7657253	test: 0.7721019	test1: 0.7882353	best: 0.7882353 (34)	total: 519ms	remaining: 12.1s
60:	learn: 0.7682458	test: 0.7732659	test1: 0.7882353	best: 0.7882353 (34)	total: 743ms	remaining: 11.4s
80:	learn: 0.7717339	test: 0.7769964	test1: 0.7976879	best: 0.7976879 (80)	total: 986ms	remaining: 11.2s
100:	learn: 0.7742653	test: 0.7805332	test1: 0.7976879	best: 0.7976879 (80)	total: 1.27s	remaining: 11.3s
120:	learn: 0.7761004	test: 0.7809789	test1: 0.8045977	best: 0.8045977 (104)	total: 1.64s	remaining: 11.9s
140:	learn: 0.7778837	test: 0.7807371	test1: 0.8045977	best: 0.8045977 (104)	total: 2.03s	remaining: 12.4s
160:	learn: 0.7794188	test: 0.7799814	test1: 0.7976879	best: 0.8045977 (104)	total: 2.36s	remaining: 12.3s
180:	learn: 0.7819740	test: 0.7814529	test1: 0.8045977	best: 0.8045977 (104)	total: 2.69s	remaining: 12.2s
200:	learn: 0.7831325	test: 0.7811968	test1: 0

20:	learn: 0.7550303	test: 0.7665670	test1: 0.7450980	best: 0.7450980 (16)	total: 268ms	remaining: 12.5s
40:	learn: 0.7647715	test: 0.7744244	test1: 0.7483871	best: 0.7532468 (33)	total: 500ms	remaining: 11.7s
60:	learn: 0.7687410	test: 0.7768775	test1: 0.7483871	best: 0.7532468 (33)	total: 705ms	remaining: 10.8s
80:	learn: 0.7709693	test: 0.7811489	test1: 0.7643312	best: 0.7643312 (72)	total: 919ms	remaining: 10.4s
100:	learn: 0.7729807	test: 0.7812597	test1: 0.7612903	best: 0.7643312 (72)	total: 1.19s	remaining: 10.6s
120:	learn: 0.7731975	test: 0.7822319	test1: 0.7612903	best: 0.7643312 (72)	total: 1.48s	remaining: 10.7s
140:	learn: 0.7747776	test: 0.7810559	test1: 0.7612903	best: 0.7643312 (72)	total: 1.76s	remaining: 10.8s
160:	learn: 0.7755556	test: 0.7816128	test1: 0.7612903	best: 0.7643312 (72)	total: 2.13s	remaining: 11.1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7643312102
bestIteration = 72

Shrink model to first 73 iterations.
err:  0.77439024390

...

20:	learn: 0.7580850	test: 0.7564800	test1: 0.7542857	best: 0.7586207 (16)	total: 236ms	remaining: 11s
40:	learn: 0.7650535	test: 0.7709887	test1: 0.7664671	best: 0.7664671 (23)	total: 461ms	remaining: 10.8s
60:	learn: 0.7685200	test: 0.7727201	test1: 0.7664671	best: 0.7664671 (23)	total: 731ms	remaining: 11.2s
80:	learn: 0.7708433	test: 0.7739791	test1: 0.7664671	best: 0.7664671 (23)	total: 1.03s	remaining: 11.7s
100:	learn: 0.7731599	test: 0.7757423	test1: 0.7664671	best: 0.7664671 (23)	total: 1.38s	remaining: 12.3s
120:	learn: 0.7765672	test: 0.7800093	test1: 0.7602339	best: 0.7692308 (108)	total: 1.79s	remaining: 13s
140:	learn: 0.7798266	test: 0.7819502	test1: 0.7630058	best: 0.7692308 (108)	total: 2.16s	remaining: 13.2s
160:	learn: 0.7817579	test: 0.7822581	test1: 0.7630058	best: 0.7692308 (108)	total: 2.53s	remaining: 13.2s
180:	learn: 0.7834726	test: 0.7825143	test1: 0.7630058	best: 0.7692308 (108)	total: 2.84s	remaining: 12.9s
200:	learn: 0.7848938	test: 0.7822581	test1: 0.763

20:	learn: 0.7606609	test: 0.7647619	test1: 0.7284768	best: 0.7368421 (13)	total: 238ms	remaining: 11.1s
40:	learn: 0.7701981	test: 0.7669173	test1: 0.7368421	best: 0.7368421 (13)	total: 546ms	remaining: 12.8s
60:	learn: 0.7742646	test: 0.7700893	test1: 0.7450980	best: 0.7450980 (51)	total: 890ms	remaining: 13.7s
80:	learn: 0.7766410	test: 0.7712377	test1: 0.7450980	best: 0.7450980 (51)	total: 1.15s	remaining: 13s
100:	learn: 0.7786524	test: 0.7726550	test1: 0.7450980	best: 0.7450980 (51)	total: 1.47s	remaining: 13.1s
120:	learn: 0.7804033	test: 0.7738965	test1: 0.7402597	best: 0.7450980 (51)	total: 1.69s	remaining: 12.3s
140:	learn: 0.7815694	test: 0.7761619	test1: 0.7402597	best: 0.7450980 (51)	total: 2.01s	remaining: 12.3s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7450980392
bestIteration = 51

Shrink model to first 52 iterations.
err:  0.7592592592592593
Learning rate set to 0.094723
0:	learn: 0.5380117	test: 0.5423865	test1: 0.5692308	best: 0.5692308 (0)

20:	learn: 0.7581761	test: 0.7656225	test1: 0.8242424	best: 0.8313253 (15)	total: 224ms	remaining: 10.4s
40:	learn: 0.7637232	test: 0.7664000	test1: 0.8447205	best: 0.8447205 (26)	total: 446ms	remaining: 10.4s
60:	learn: 0.7670274	test: 0.7693043	test1: 0.8447205	best: 0.8447205 (26)	total: 719ms	remaining: 11.1s
80:	learn: 0.7687172	test: 0.7695977	test1: 0.8447205	best: 0.8447205 (26)	total: 972ms	remaining: 11s
100:	learn: 0.7700382	test: 0.7722301	test1: 0.8518519	best: 0.8518519 (88)	total: 1.16s	remaining: 10.3s
120:	learn: 0.7734772	test: 0.7760727	test1: 0.8363636	best: 0.8518519 (88)	total: 1.45s	remaining: 10.5s
140:	learn: 0.7773728	test: 0.7770840	test1: 0.8363636	best: 0.8518519 (88)	total: 1.76s	remaining: 10.7s
160:	learn: 0.7793071	test: 0.7783144	test1: 0.8363636	best: 0.8518519 (88)	total: 2.09s	remaining: 10.9s
180:	learn: 0.7822785	test: 0.7771019	test1: 0.8363636	best: 0.8518519 (88)	total: 2.38s	remaining: 10.8s
Stopped by overfitting detector  (100 iterations wai

20:	learn: 0.7587413	test: 0.7668557	test1: 0.7515924	best: 0.7594937 (16)	total: 194ms	remaining: 9.05s
40:	learn: 0.7657886	test: 0.7731119	test1: 0.7577640	best: 0.7594937 (16)	total: 383ms	remaining: 8.95s
60:	learn: 0.7689606	test: 0.7734894	test1: 0.7701863	best: 0.7701863 (59)	total: 648ms	remaining: 9.97s
80:	learn: 0.7721700	test: 0.7732448	test1: 0.7701863	best: 0.7701863 (59)	total: 905ms	remaining: 10.3s
100:	learn: 0.7754130	test: 0.7782283	test1: 0.7710843	best: 0.7777778 (82)	total: 1.11s	remaining: 9.86s
120:	learn: 0.7771774	test: 0.7778124	test1: 0.7710843	best: 0.7777778 (82)	total: 1.38s	remaining: 10.1s
140:	learn: 0.7788202	test: 0.7791115	test1: 0.7710843	best: 0.7777778 (82)	total: 1.67s	remaining: 10.2s
160:	learn: 0.7817517	test: 0.7794232	test1: 0.7710843	best: 0.7777778 (82)	total: 2.04s	remaining: 10.6s
180:	learn: 0.7843572	test: 0.7794920	test1: 0.7710843	best: 0.7777778 (82)	total: 2.4s	remaining: 10.9s
Stopped by overfitting detector  (100 iterations wa

20:	learn: 0.7575709	test: 0.7682349	test1: 0.7500000	best: 0.7530864 (11)	total: 204ms	remaining: 9.51s
40:	learn: 0.7674234	test: 0.7751377	test1: 0.7625000	best: 0.7625000 (39)	total: 412ms	remaining: 9.64s
60:	learn: 0.7681044	test: 0.7754011	test1: 0.7672956	best: 0.7672956 (43)	total: 625ms	remaining: 9.63s
80:	learn: 0.7684934	test: 0.7765706	test1: 0.7594937	best: 0.7672956 (43)	total: 860ms	remaining: 9.76s
100:	learn: 0.7715428	test: 0.7766410	test1: 0.7594937	best: 0.7672956 (43)	total: 1.08s	remaining: 9.6s
120:	learn: 0.7737506	test: 0.7781784	test1: 0.7672956	best: 0.7672956 (43)	total: 1.32s	remaining: 9.63s
140:	learn: 0.7782906	test: 0.7784225	test1: 0.7594937	best: 0.7672956 (43)	total: 1.59s	remaining: 9.72s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7672955975
bestIteration = 43

Shrink model to first 44 iterations.
err:  0.7716049382716049


In [19]:
np.mean(errcb)

0.7767720820101862

In [20]:
#model.fit(X, y,cat_features=categorical_features_indices, early_stopping_rounds=100)

In [21]:
#preds = model.predict(test)

In [22]:
#pd.DataFrame({'ID':test_id, 'Won_Championship':preds.astype(int)}).to_csv('win27.csv', index=False)

In [23]:
preds = np.mean(y_pred_totcb, 0)

In [24]:
pred = pd.DataFrame(preds, columns=['Won_Championship'])

In [25]:
pred['Won_Championship'].value_counts()

0.000    1578
1.000    1505
0.950      71
0.925      41
0.025      27
0.225      25
0.475      24
0.525      22
0.150      22
0.350      18
0.975      17
0.600      15
0.800      13
0.500      13
0.900      12
0.250      12
0.050      11
0.125      11
0.275       8
0.825       7
0.300       6
0.775       5
0.075       4
0.675       3
0.575       3
0.625       3
0.375       3
0.700       3
0.325       3
0.100       3
0.175       2
0.850       2
0.750       2
0.200       2
0.875       2
0.725       1
0.450       1
Name: Won_Championship, dtype: int64

In [26]:
def thresh(x):
    if x <= 0.2:#0.2
        return 0
    else:
        return 1

In [27]:
pred['Won_Championship'] = pred['Won_Championship'].apply(thresh)

In [28]:
pred['Won_Championship'].value_counts()

1    1844
0    1656
Name: Won_Championship, dtype: int64

In [29]:
pd.DataFrame({'ID':test_id, 'Won_Championship':pred['Won_Championship'].astype(int)}).to_csv('winc.csv', index=False)

In [ ]:
LGBMClassifier(num_leaves= 5, max_depth=5, 
                         random_state=314,
                         metric='None', 
                         n_jobs=4, class_weight='balanced',
                         n_estimators=3000,
                         #colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.015)